
# 🌳 Cours & TPs : Random Forest et Arbres de Décision

## 1️⃣ Introduction – Le surapprentissage d’un arbre de décision

Un arbre de décision non limité peut **surapprendre** les données, c’est-à-dire mémoriser les points d’apprentissage sans généraliser correctement.



## 🧪 TP 1 – Arbre de Décision vs Régression Linéaire

Nous allons simuler une relation simple `y = x + bruit` et entraîner :
- un **arbre de décision** (sans limitation)
- une **régression linéaire**


In [ ]:
# Installation des bibliothèques
!pip install pandas numpy scikit-learn matplotlib

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

np.random.seed(0)
X = np.arange(1, 11).reshape(-1, 1)
y = X.flatten() + np.random.normal(scale=0.5, size=X.shape[0])

tree = DecisionTreeRegressor()
tree.fit(X, y)

linear = LinearRegression()
linear.fit(X, y)

X_test = np.linspace(1, 10, 100).reshape(-1, 1)
y_tree = tree.predict(X_test)
y_linear = linear.predict(X_test)

plt.figure(figsize=(10, 6))
plt.scatter(X, y, color='black', label="Données")
plt.plot(X_test, y_tree, color='red', label="Arbre (overfitting)")
plt.plot(X_test, y_linear, color='blue', label="Régression linéaire")
plt.title("Surapprentissage : Arbre vs Régression")
plt.xlabel("x"); plt.ylabel("y")
plt.legend(); plt.grid(True); plt.show()



### ❓ Questions :
1. Pourquoi la courbe rouge est-elle irrégulière ?
2. Quel modèle suit la tendance globale ?
3. Lequel serait plus fiable pour prédire un nouveau point ?



## 📘 2️⃣ Cours – Algorithme Random Forest

Random Forest est un **ensemble d’arbres de décision**, chacun entraîné sur un échantillon **aléatoire** du jeu de données, avec des variables sélectionnées aléatoirement à chaque nœud.

### Étapes :
1. **Bootstrap** : échantillonnage aléatoire avec répétition
2. **Sélection aléatoire de variables**
3. **Agrégation** :
   - Vote majoritaire (classification)
   - Moyenne (régression)

### ✅ Avantages :
- Moins de surapprentissage
- Plus stable et robuste

### ⚠️ Inconvénients :
- Moins interprétable
- Temps de calcul plus long



## 🧪 TP 2 – Classification : Reconnaître un animal

Nous allons utiliser un `RandomForestClassifier` pour prédire si un animal est un **mammifère**, un **poisson** ou un **oiseau**, à partir de ses caractéristiques.


In [ ]:

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

data = pd.DataFrame({
    'taille': ['petit', 'moyen', 'grand', 'petit', 'grand', 'moyen'],
    'poils': ['oui', 'non', 'non', 'oui', 'non', 'oui'],
    'aquatique': ['non', 'oui', 'non', 'non', 'oui', 'non'],
    'classe': ['mammifère', 'poisson', 'oiseau', 'mammifère', 'poisson', 'mammifère']
})

enc_taille = LabelEncoder()
enc_poils = LabelEncoder()
enc_aqua = LabelEncoder()
enc_classe = LabelEncoder()

data['taille'] = enc_taille.fit_transform(data['taille'])
data['poils'] = enc_poils.fit_transform(data['poils'])
data['aquatique'] = enc_aqua.fit_transform(data['aquatique'])
data['classe'] = enc_classe.fit_transform(data['classe'])

X = data[['taille', 'poils', 'aquatique']]
y = data['classe']

# Algorithme Random Forest : n_estimators est le nombre d'arbres
clf = RandomForestClassifier(n_estimators=10, random_state=0)
clf.fit(X, y)

# Test d'un nouvel animal 
nouvel_animal = [[enc_taille.transform(['moyen'])[0],
                  enc_poils.transform(['oui'])[0],
                  enc_aqua.transform(['non'])[0]]]

prediction = clf.predict(nouvel_animal)
print("Classe prédite :", enc_classe.inverse_transform(prediction)[0])


**Remarque** : le code ci-dessous permet de trouver **l'importance de chaque variable. TRES UTILE !**

In [ ]:
import matplotlib.pyplot as plt

importances = clf.feature_importances_
colonnes = X.columns

plt.bar(colonnes, importances)
plt.title("Importance des variables")
plt.ylabel("Importance")
plt.grid(True)
plt.show()


### ❓ Questions :
1. Quelle classe est prédite ?
2. Tester avec un petit animal, sans poils et non aquatique.
3. Pourquoi Random Forest est-elle plus fiable qu’un arbre seul ?



## 🔁 Activité complémentaire – Influence de `n_estimators`

Testons l'effet du nombre d'arbres sur la **précision** et le **temps de calcul**.


In [ ]:

from sklearn.ensemble import RandomForestClassifier
import time

for n in [1, 5, 10, 100]:
    clf = RandomForestClassifier(n_estimators=n, random_state=0)
    start = time.time()
    clf.fit(X, y)
    score = clf.score(X, y)
    duration = time.time() - start
    print(f"{n} arbres → Score : {score:.2f} | Temps : {duration:.4f} sec")



### 📈 Visualisation de l’évolution du score


In [ ]:
# Importation des bibliothèques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

# Étape 1 : Génération d'un grand jeu de données
# 10000 échantillons, 15 variables, 3 classes
X, y = make_classification(n_samples=10000,
                           n_features=15,
                           n_informative=6,
                           n_redundant=2,
                           n_classes=3,
                           random_state=42) # Permet d'avoir le même jeu de données, on peut mettre un nombre aléatoire. 
                                            # Mettre 0 sinon.

# Étape 2 : Liste des tailles de forêt à tester
valeurs = [1, 5, 10, 20, 50, 100, 200, 300]
scores = []

# Étape 3 : Boucle sur chaque valeur de n_estimators
for n in valeurs:
    start = time.time()
    clf = RandomForestClassifier(n_estimators=n, random_state=0)
    clf.fit(X, y)
    score = clf.score(X, y)  # Score sur les données d'entraînement
    scores.append(score)
    duration = time.time() - start
    print(f"{n} arbres → Score : {score:.2f} | Temps : {duration:.4f} sec")

# Étape 4 : Visualisation du score
plt.figure(figsize=(8, 5))
plt.plot(valeurs, scores, marker='o')
plt.title("Évolution du score selon le nombre d’arbres (n_estimators)")
plt.xlabel("Nombre d’arbres")
plt.ylabel("Score sur l’ensemble d’apprentissage")
plt.grid(True)
plt.show()



### ❓ Questions :
1. À partir de combien d’arbres le score se stabilise-t-il ?
2. Le temps de calcul augmente-t-il beaucoup ?
3. Quelle valeur serait un bon compromis ?


**Remarque** : il existe `RandomForestRegressor` qui permet de prédire une **valeur** plutôt qu'une classe. Je vous laisse chercher :).